## Section 2.4: Incremental Implementation
So we've seen the benefits of $\epsilon$-greedy strategies, but let's take a second to deal with the implementation. 

Our naive approach to estimating the rewards of each action requires $O(n)$ memory and computation, because, assuming $t=n$, we need to average the last $n$ rewards (recall that $Q_t$, shorthard for $Q_t(a)$ is the estimate of some action's true reward at time $t$):
$$
Q_n = \frac{R_1 + ... + R_{n-1}}{n-1}
$$
That could end up being a lot of additions and rewards stored! In fact, we could even encounter numerical instability issues if the sum of rewards got very large. So, we find a way to calculate $Q_n$ incrementally:
$$
\begin{align}
Q_n &= \frac{R_1 + ... + R_{n-1}}{n-1} \\
&= \frac{R_1 + ... + R_{n-2}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-2} \frac{R_1 + ... + R_{n-2}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} \frac{R_1 + ... + R_{n-2}}{n-2} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} Q_{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{n-2}{n-1} Q_{n-1} + \frac{R_{n-1}}{n-1} \\
&= \frac{(n-1)Q_{n-1} - Q_{n-1}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= Q_{n-1} - \frac{Q_{n-1}}{n-1} + \frac{R_{n-1}}{n-1} \\
&= Q_{n-1} + \frac{1}{n-1}[R_{n-1} - Q_{n-1}] \\
\end{align}
$$
Wowza! So this gives us a recursive formula for computing the reward in constant time and memory. 

It also looks a bit like gradient descent, with a step size of $1/t$. In fact, it is! It's clearer if we look at the case for $t=n+1$ and rearrange a bit:
$$
Q_{n+1} = Q_n + \frac{1}{n}[R_n - Q_n]
$$
Sutton & Barto make this clear in the book:
$$
\text{NewEstimate} = \text{OldEstimate} + \text{StepSize}[\text{NewEstimate} - \text{OldEstimate}].
$$
Of course, this begs the question: what happens if we change the step size?

## Section 2.5: Tracking a Nonstationary Problem
So, using weights of $1/t$ is great for ensuring convergence to the true mean rewards for each action, but only if the rewards don't change. In applied RL, the rewards distributions are always changing. So, we'll want a step size that doesn't decrease as time goes on. Instead, we want it to weigh the most recent evidence most strongly. So, we can generalize, and let
$$
\alpha_t(a) := 1/t
$$
so that
$$
Q_{n+1} = Q_n + \alpha_n(a)[R_n - Q_n].
$$

Let's compute the weight at each step if $\alpha$ is constant in the update function. This creates an exponential moving average of past rewards:
$$
\begin{align}
Q_{n+1} &= Q_n + \alpha[R_n - Q_n] \\
&= \alpha R_n - (1 - \alpha) Q_n \\
&= \alpha R_n - (1 - \alpha) [Q_{n-1} + \alpha(R_{n-1} - Q_{n-1})] \\
&= \alpha R_n - (1 - \alpha) [\alpha R_{n-1} + (1 - \alpha) Q_{n-1}] \\
&= \alpha R_n - (1 - \alpha)\alpha R_{n-1} - (1 - \alpha)^2 Q_{n-1} \\
&= \alpha R_n - (1 - \alpha)\alpha R_{n-1} - (1 - \alpha)^2 [\alpha R_{n-2} + (1-\alpha)Q_{n-2}] \\
&= \alpha R_n - (1 - \alpha)\alpha R_{n-1} - (1 - \alpha)^2\alpha R_{n-2} + (1-\alpha)^3 Q_{n-2} \\
&= \alpha R_n - (1 - \alpha)\alpha R_{n-1} - (1 - \alpha)^2\alpha R_{n-2} + \cdots + (1 - \alpha)^{n-1}\alpha R_{1} + (1-\alpha)^n Q_1 \\
&= \sum_{i=1}^{n} (1-\alpha)^{n-i}\alpha R_n + (1-\alpha)^n Q_1 \\
\end{align}
$$
We can check that the weights sum to one. If $0 < \alpha \leq 1$, we can transform one side into a geometric series and sum it to show that this is indeed true (intermediately, we let $r = 1 - \alpha$):
$$
\sum_{i=1}^n (1-\alpha)^{n-i}\alpha + (1-\alpha)^n = 1
\Leftrightarrow \sum_{i=0}^{n-1} (1-\alpha)^{i} = [1 - (1 - \alpha)^n]/\alpha
$$
$$
\sum_{i=0}^{n-1} (1-\alpha)^{i} = \sum_{i=0}^{n-1} r^{i} = \frac{1 - r^n}{1 - r} = \frac{1 - (1 - \alpha)^n}{1 - (1 - \alpha)}
$$

Let's take a look at the weightings for some various values of $n$:

In [28]:
import numpy as np
from bokeh.plotting import figure, show, output_file

alpha = 0.2
n = 20
weights = np.zeros(n+1)
weights.fill(1-alpha)
powers = np.arange(n+1)
weights = np.power(weights, powers)
weights = weights * alpha
weights[-1] = weights[-1] * 1/alpha
print(weights)

p = figure()
p.line(np.arange(n+1), weights)
p.square(np.arange(n+1), weights)
output_file("plot.html", title="weights")
show(p)

[0.25       0.1875     0.140625   0.10546875 0.07910156 0.05932617
 0.04449463 0.03337097 0.02502823 0.01877117 0.01407838 0.01055878
 0.00791909 0.00593932 0.00445449 0.00334087 0.00250565 0.00187924
 0.00140943 0.00105707 0.00317121]


#### Exercise 2.4
If the weights aren't constant, what form does the weighting function take?
$$
\begin{align}
Q_{n+1} &= Q_n + \alpha_n[R_n - Q_n] \\
&= \alpha_n R_n - (1 - \alpha_n) Q_n \\
&= \alpha_n R_n - (1 - \alpha_n) [Q_{n-1} + \alpha_{n-1}(R_{n-1} - Q_{n-1})] \\
&= \alpha_n R_n - (1 - \alpha_n) [\alpha_{n-1} R_{n-1} + (1 - \alpha_{n-1}) Q_{n-1}] \\
&= \alpha_n R_n - (1 - \alpha_n)\alpha_{n-1} R_{n-1} - (1 - \alpha_n)(1 - \alpha_{n-1}) Q_{n-1} \\
&= \alpha_n R_n - (1 - \alpha_n)\alpha_{n-1} R_{n-1} - (1 - \alpha_n)(1 - \alpha_{n-1}) [\alpha_{n-2} R_{n-2} + (1-\alpha_{n-2})Q_{n-2}] \\
&= \alpha_n R_n - (1 - \alpha_n)\alpha_{n-1} R_{n-1} - (1 - \alpha_n)(1 - \alpha_{n-1})\alpha_{n-2} R_{n-2} - (1 - \alpha_n)(1 - \alpha_{n-1})(1-\alpha_{n-2})Q_{n-2} \\
&= \alpha_n R_n - (1 - \alpha_n)\alpha_{n-1} R_{n-1} - (1 - \alpha_n)(1 - \alpha_{n-1})\alpha_{n-2} R_{n-2} 
- \cdots - (1 - \alpha_n)\cdots(1 - \alpha_{2})\alpha_{1} R_{1} 
+ (1 - \alpha_n)\cdots(1 - \alpha_{1}) Q_1 \\
&= \alpha_n R_n - \sum_{i=1}^{n-1} \alpha_{n-i} \bigg [ \prod_{j=0}^{i-1} (1 - \alpha_{n-j}) \bigg ] R_{n-i} 
+ \bigg [ \prod_{j=0}^{n-1} (1 - \alpha_{n-j}) \bigg ] Q_1
\end{align}
$$
So what does this look like?

In [45]:
import numpy as np
from bokeh.plotting import figure, show, output_file

alphas = np.random.rand(21)
print(alphas)
n = 20
weights = np.zeros(n+1)
factor = 1
for i in range(len(weights)):
    if i==0:
        weights[i] = alphas[i]
    else:
        factor *= 1 - alphas[i-1]
        weights[i] = alphas[i] * factor
factor *= 1 - alphas[i]
weights[n] = factor
print(weights)

p = figure()
p.line(np.arange(n+1), weights)
p.square(np.arange(n+1), weights)
output_file("plot.html", title="weights")
show(p)

[0.3397359  0.07980063 0.14261644 0.61243859 0.80163521 0.83573659
 0.73297388 0.16338082 0.80348344 0.29084813 0.55973334 0.10450719
 0.96169572 0.41221409 0.76668851 0.8645365  0.13788867 0.33375344
 0.19430304 0.66873095 0.32171173]
[3.39735902e-01 5.26894931e-02 8.66501291e-02 3.19034251e-01
 1.61842313e-01 3.34695055e-02 4.82180012e-03 2.86995780e-04
 1.18080773e-03 8.39977575e-05 1.14636219e-04 9.42329183e-06
 7.76526464e-05 1.27493695e-06 1.39381138e-06 3.66694589e-07
 7.92267844e-09 1.65322721e-08 6.41241145e-09 1.77813577e-08
 5.97459776e-09]
